# 📘 Lectura desde la Web (APIs) y JSON → pandas  
**Con persistencia en Parquet/Pickle y opciones SQLite/MongoDB**  

> Notebook orientado a *Ciencia de Datos*: consumir APIs web (JSON), normalizar y analizar con **pandas**, y **persistir** eficientemente para reutilización y performance.

## 🎯 Objetivos
- Entender el flujo **API → JSON → Python → DataFrame**.
- Consumir datos desde la web con **requests** (GET, headers, paginación, manejo de errores y rate limits).
- Transformar JSON (incluyendo **anidado**) a **pandas DataFrame** con `json_normalize`.
- Realizar limpieza: tipado, faltantes, selección de columnas, arrays con `explode`.
- Persistir resultados en **Parquet** (columnar) y **Pickle** (Python-only, con nota de seguridad).
- (Opcional) Guardar/leer en **SQLite** y **MongoDB**.
- Medir performance y aplicar buenas prácticas.


## 🧠 Flujo general (mapa mental)

```
API (HTTP) ──► respuesta JSON (texto) ──► response.json() ──► dict/list (Python)
     │
     └──► status_code, headers, paginación, rate limits

dict/list ──► pandas.json_normalize(...) ──► DataFrame ──► selección/limpieza
                                                        └─► persistencia: Parquet / Pickle / SQLite / Mongo
```


## 🧾 JSON rápido (equivalencias)
| JSON                          | Python            | Ejemplo                                 |
|------------------------------|-------------------|-----------------------------------------|
| `{ "clave": "valor" }`       | `dict`            | `{"id": 1, "title": "Mouse"}`           |
| `[ { ... }, { ... } ]`       | `list`            | `[{"id":1},{"id":2}]`                   |
| `"texto"`                    | `str`             | `"hola"`                                |
| `123`, `45.6`                | `int` / `float`   | `42` / `3.14`                           |
| `true`, `false`              | `bool`            | `True` / `False`                        |
| `null`                       | `None`            | `None`                                  |


## 🧰 Librerías
> Ejecutá la celda de instalación si te falta algo (descomentá las líneas).

In [ ]:
# !pip install requests pandas pyarrow fastparquet pymongo matplotlib
import json, time, math, sys, os
import requests
import pandas as pd
from pandas import json_normalize
print('Versions → pandas:', pd.__version__)


## 1) GET a una API pública (con manejo de errores) + **fallback offline**
Usamos `https://dummyjson.com/products`. Si no hay internet, usamos un **JSON de ejemplo** embebido.

In [ ]:
URL = 'https://dummyjson.com/products'

def fetch_json(url: str, timeout=10):
    try:
        r = requests.get(url, timeout=timeout)
        if r.status_code != 200:
            raise RuntimeError(f'HTTP {r.status_code}: {r.text[:120]}')
        return r.json()
    except Exception as e:
        print('⚠️ No se pudo descargar desde la web. Usando fallback offline.\nMotivo:', e)
        # Fallback mínimo similar a dummyjson
        return {
            'products': [
                {'id':1,'title':'Phone','brand':'Acme','price':199.99,'rating':4.5,'tags':['tech','mobile']},
                {'id':2,'title':'Laptop','brand':'Acme','price':999.0,'rating':4.7,'tags':['tech','computer']},
                {'id':3,'title':'Headphones','brand':'Sonic','price':79.5,'rating':4.2,'tags':['audio','tech']}
            ],
            'total': 3,
            'skip': 0,
            'limit': 3
        }

data = fetch_json(URL)
type(data), list(data)[:5], (len(data['products']) if 'products' in data else None)


### Inspección rápida del JSON

In [ ]:
print('Claves de nivel 1:', list(data.keys()))
if 'products' in data:
    print('Total productos reportado:', data.get('total'))
    print('Ejemplo de primer producto:')
    print(json.dumps(data['products'][0], indent=2, ensure_ascii=False))


## 2) De JSON a DataFrame con `json_normalize`
- Si el JSON es **lista de objetos**: `pd.DataFrame(lista)`
- Si el JSON es **anidado**: `json_normalize` ayuda a **aplanar**.

In [ ]:
items = data['products'] if 'products' in data else data
df = pd.DataFrame(items)
df.head()


In [ ]:
# Si existieran subcampos anidados, json_normalize los aplanaría con nombres jerárquicos 'a.b.c'
df_flat = json_normalize(items, sep='.')
df_flat.head()


## 3) Selección y renombre de columnas
Elegimos un subconjunto **mínimo viable** para análisis.

In [ ]:
keep = [c for c in ['id','title','brand','price','rating','tags'] if c in df_flat.columns]
df_view = df_flat[keep].copy()
df_view = df_view.rename(columns={'title':'name'})
df_view.head()


## 4) Tipado y valores faltantes
- Convertimos a numérico donde corresponde.
- Calculamos % de faltantes por columna.
- Creamos banderas de faltantes si hace falta.

In [ ]:
for col in ['price','rating']:
    if col in df_view.columns:
        df_view[col] = pd.to_numeric(df_view[col], errors='coerce')
na_pct = df_view.isna().mean().round(3)
na_pct


In [ ]:
# Bandera de faltantes opcional (tipo entero 0/1)
for col in df_view.columns:
    df_view[f'{col}_isna'] = df_view[col].isna().astype('Int8')
df_view.head()


## 5) Manejo de listas dentro del JSON (`explode`)
Cuando hay arrays (ej. `tags`), `explode` crea una fila por elemento.

In [ ]:
if 'tags' in df_view.columns:
    df_tags = df_view[['id','name','tags']].explode('tags', ignore_index=True)
    df_tags.head()
else:
    print("No hay columna 'tags' en este dataset.")


## 6) Visualización rápida (opcional)
> Requiere `matplotlib`. No seteamos estilos ni colores.

In [ ]:
import matplotlib.pyplot as plt

if 'brand' in df_view.columns:
    counts = df_view['brand'].value_counts()
    plt.figure()
    counts.plot(kind='bar', title='Cantidad por marca')
    plt.show()
else:
    print("No hay columna 'brand' para graficar.")


## 7) Paginación y rate limits (patrón general)
Muchas APIs devuelven resultados en páginas. Patrón típico:

In [ ]:
def fetch_paginated(base_url, page_param='page', start=1, max_pages=3, per_page_param=None, per_page=50, delay=0.5):
    """Ejemplo genérico de paginación:
    - base_url: 'https://api.ejemplo.com/items'
    - page_param: 'page' o 'offset' según la API
    - per_page_param: nombre del parámetro de tamaño de página si aplica
    - delay: para respetar límites de rate (evitar bloqueos)
    """
    all_items = []
    for p in range(start, start+max_pages):
        params = {page_param: p}
        if per_page_param:
            params[per_page_param] = per_page
        r = requests.get(base_url, params=params, timeout=10)
        if r.status_code != 200:
            print('Página', p, 'HTTP', r.status_code)
            break
        payload = r.json()
        # ⚠️ Adaptar a cómo venga la lista en tu API, p.ej. payload['results'] o payload['data']
        items = payload.get('results') or payload.get('data') or payload.get('products') or []
        all_items.extend(items)
        time.sleep(delay)
        if not items:
            break
    return all_items

# Ejemplo (no ejecutable si la API no existe): 
# items = fetch_paginated('https://dummyjson.com/products')  # dummyjson usa otros parámetros
# len(items)


## 8) Autenticación (Bearer/API-Key)
Usá headers y nunca subas tu token al repo público.

In [ ]:
API_URL = 'https://api.ejemplo.com/data'
TOKEN = os.getenv('API_TOKEN')  # definí API_TOKEN en tu entorno

def get_with_auth(url):
    if not TOKEN:
        raise RuntimeError('Definí la variable de entorno API_TOKEN con tu token.')
    headers = {'Authorization': f'Bearer {TOKEN}'}
    r = requests.get(url, headers=headers, timeout=10)
    r.raise_for_status()
    return r.json()

# data_auth = get_with_auth(API_URL)
# data_auth


## 9) Persistencia eficiente: **Parquet**
- **Columnar**, **comprimido**, **portable** (Python, R, Spark).
- Permite **leer solo columnas** necesarias.

In [ ]:
# Guardar
df_view.to_parquet('products.parquet', engine='pyarrow', compression='snappy')

# Leer solo algunas columnas
subset_cols = [c for c in ['id','name','price'] if c in df_view.columns]
subset = pd.read_parquet('products.parquet', columns=subset_cols if subset_cols else None)
subset.head()


## 10) Persistencia **Pickle** (solo Python)
- Guarda/recupera objetos de Python tal cual.
- Seguridad: **no** abras pickles de fuentes no confiables.

In [ ]:
df_view.to_pickle('products.pkl')
df_back = pd.read_pickle('products.pkl')
df_back.head(3)


## 11) SQLite (SQL local en archivo `.db`)
Ideal para consultas rápidas sin montar un servidor.

In [ ]:
import sqlite3

con = sqlite3.connect('products.db')
df_view.to_sql('products', con, if_exists='replace', index=False)
q = 'SELECT id, name, price FROM products ORDER BY price DESC LIMIT 5'
top5 = pd.read_sql(q, con)
con.close()
top5


## 12) MongoDB (documentos JSON)
Útil para estructuras **muy anidadas** o cambiantes (*schema-less*).

In [ ]:
# from pymongo import MongoClient  # requeriría servidor de MongoDB
# client = MongoClient('mongodb://localhost:27017/')
# db = client['ejemplo_db']
# docs = df_view.to_dict(orient='records')
# db.libros.insert_many(docs)          # usar colección adecuada (ej. 'libros')
# db.libros.find_one()


## 13) Validaciones básicas de esquema y valores
Control mínimo para garantizar calidad antes de persistir o publicar.

In [ ]:
expected_cols = set(['id','name','brand','price','rating']) & set(df_view.columns)
missing = expected_cols - set(df_view.columns)
print('Columnas esperadas presentes:', expected_cols)
print('Faltan columnas:', missing)

if 'price' in df_view.columns:
    neg = (df_view['price'] < 0).sum()
    print('Precios negativos:', neg)


## 14) Medición de performance: Parquet (todas vs. subset de columnas)

In [ ]:
import time

start = time.perf_counter()
_ = pd.read_parquet('products.parquet')  # todas las columnas
t_all = time.perf_counter() - start

cols = [c for c in ['id','name','price'] if c in df_view.columns]
start = time.perf_counter()
_ = pd.read_parquet('products.parquet', columns=cols if cols else None)
t_subset = time.perf_counter() - start

print(f'Leer todo: {t_all:.4f}s | Solo {len(cols)} cols: {t_subset:.4f}s')


## ✅ Checklist rápido
- [ ] `requests.get(URL)` con `timeout` y chequeo de `status_code`.
- [ ] `response.json()` → dict/list de Python.
- [ ] `json_normalize` para **aplanar** estructuras anidadas.
- [ ] Seleccionar columnas clave (subconjunto mínimo viable).
- [ ] Tipar numéricos con `pd.to_numeric` y revisar faltantes.
- [ ] Manejar arrays con `explode`.
- [ ] Guardar en **Parquet** y, si hace falta, en **Pickle** (seguro).
- [ ] (Opcional) Persistir en **SQLite/Mongo** según el caso.
- [ ] Medir tiempos de lectura para elegir la mejor estrategia.
- [ ] No subir **tokens** ni **pickles** no verificados a repos públicos.

## 🧭 Buenas prácticas
- Documentá el contrato de la API (estructura de respuesta, paginación, límites, autenticación).
- Usá variables de entorno para secretos (`os.getenv`).
- Versioná tus *notebooks* y exportá datasets intermedios en Parquet.
- Validá esquema (columnas obligatorias) y reglas de negocio antes de persistir.
